In [2]:
import os
import sys
import pandas as pd
import gzip
import gemmi
import config
from tqdm import tqdm
from datetime import datetime
import wget
from calculate import printer,detect_plevin
from addH import addH

ring_atoms_dict = {
    'TRP': ['CD2', 'CE2', 'CE3', 'CZ2', 'CZ3', 'CH2'],
    'TYR': ['CD1', 'CD2', 'CE1', 'CE2', 'CZ', 'CG'],
    'PHE': ['CD1', 'CD2', 'CE1', 'CE2', 'CZ', 'CG'],
    'HIS': ['CE1', 'ND1', 'NE2', 'CG', 'CD2']}
trp_A_dict = {
    'TRP': ['CD1', 'CD2', 'NE1', 'CG', 'CE2']}
printer.print_xhpi()

CC     CC    HH   HH             PPPPPPP    II  
 CC   CC     HH   HH             PP   PP    II 
  NN NN      HH   HH     York    PP   PP    II 
   NN        HHHHHHH    ------   PPPPPPP    II 
  OO OO      HH   HH             PP         II 
 OO   OO     HH   HH             PP         II 
SS     SS    HH   HH             PP         II 


In [4]:
input_directory = "C:/Users/Sean/Desktop/xhpi_for_database/test_addH/"

gz_files = []
for dirpath, _, filenames in os.walk(input_directory):
    for filename in filenames:
        if filename.endswith(".cif.gz"):
            # 将文件的完整路径添加到列表中
            gz_files.append(os.path.join(dirpath, filename))
gz_files

['C:/Users/Sean/Desktop/xhpi_for_database/test_addH/test_addH\\d8\\5d8v.cif.gz',
 'C:/Users/Sean/Desktop/xhpi_for_database/test_addH/test_addH\\e6\\6e6o.cif.gz',
 'C:/Users/Sean/Desktop/xhpi_for_database/test_addH/test_addH\\ni\\3nir.cif.gz',
 'C:/Users/Sean/Desktop/xhpi_for_database/test_addH/test_addH\\nw\\5nw3.cif.gz',
 'C:/Users/Sean/Desktop/xhpi_for_database/test_addH/test_addH\\r6\\1r6j.cif.gz',
 'C:/Users/Sean/Desktop/xhpi_for_database/test_addH/test_addH\\s2\\6s2m.cif.gz',
 'C:/Users/Sean/Desktop/xhpi_for_database/test_addH/test_addH\\uc\\1ucs.cif.gz',
 'C:/Users/Sean/Desktop/xhpi_for_database/test_addH/test_addH\\us\\1us0.cif.gz',
 'C:/Users/Sean/Desktop/xhpi_for_database/test_addH/test_addH\\vb\\2vb1.cif.gz',
 'C:/Users/Sean/Desktop/xhpi_for_database/test_addH/test_addH\\x2\\3x2m.cif.gz']

In [5]:
result = []

with tqdm(total=len(gz_files), desc="Processing files") as pbar:
    for filepath in gz_files:
        try:
            pdb_name = os.path.basename(filepath).replace('.cif.gz', '')

            # Read the gzipped CIF file
            with gzip.open(filepath, 'rb') as file:
                uncompressed_content = file.read().decode('utf-8')
                cif = gemmi.cif.read_string(uncompressed_content).sole_block()
                structure = gemmi.make_structure_from_block(cif)
                resolution = structure.resolution
                model = structure[0]

                for chain in model:
                    for residue in chain:
                        
                        if residue.name in ring_atoms_dict:
                            found_interactions = detect_plevin.detect_plevin(
                                pdb_name, resolution, model, chain, structure, residue, ring_atoms_dict)
                            result.extend(found_interactions)

                        if residue.name in trp_A_dict:
                            found_interactions = detect_plevin.detect_plevin(
                                pdb_name, resolution, model, chain, structure, residue, trp_A_dict)
                            result.extend(found_interactions)

        except Exception as e:
            print(f"Error processing file {filepath}: {e}")

        finally:
            pbar.update(1)

print(f"Detected {len(result)} XH-π interactions in total.")

# Save the calculation results to a CSV file
if result: # Only save if results were found
    df = pd.DataFrame(result)
    output_path = os.path.join(input_directory, 'xhpi_output.csv')
    df.to_csv(output_path, index=False)
    print(f'The result has been saved to {output_path}')
else:
    print("No interactions were found to save.")

Processing files: 100%|██████████| 10/10 [00:00<00:00, 49.91it/s]

Detected 115 XH-π interactions in total.
The result has been saved to C:/Users/Sean/Desktop/xhpi_for_database/test_addH/xhpi_output.csv


In [6]:
found_interactions

[{'pdb_name': '3x2m',
  'resolution': 0.64,
  'mean_b_factor': np.float64(3.801000142097473),
  'pi_chain_name': 'A',
  'pi_residue_number': 167,
  'pi_residue_name': 'TRP_A',
  'pi_center_array': array([ 0.6796, -4.4096, -6.9434]),
  'pi_normal_vector': array([-0.45322016,  0.25339462, -0.85462428]),
  'pi_b_factor': np.float64(3.59200005531311),
  'X_chain_name': 'A',
  'X_residue_num': 7,
  'X_residue_name': 'GLN',
  'X_element_name': 'C',
  'X_atom_name': 'CB',
  'X_pos': array([  0.251,  -4.304, -10.764]),
  'X_b_factor': 4.010000228881836,
  'H_atom_name': 'HB3',
  'H_pos': array([ 0.44179813, -4.49315144, -9.82154692]),
  'X_to_pi_center_distance': np.float64(3.8460152989815306),
  'XH_picenterAngle': np.float64(165.5723914428692),
  'XPCN_angle': np.float64(24.98192719385196)}]